<a href="https://colab.research.google.com/github/purvasingh96/Deep-Reinforcement-Learning/blob/master/4.%20Deep%20Q%20Networks/Credit_Card_Fraud_Detection_via_DQNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading data and Making *gym-fraud-detection* environment

In [0]:
import shutil

shutil.rmtree('/content/gym-fraud-detection')

In [0]:
!unzip gym-fraud-detection.zip

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
unzip:  cannot find or open gym-fraud-detection.zip, gym-fraud-detection.zip.zip or gym-fraud-detection.zip.ZIP.


In [0]:
os.chdir('/content/')

In [0]:
print(os.getcwd())

/content


In [0]:
!unzip gym-fraud-detection.zip

In [0]:
import os
os.chdir('gym-fraud-detection')

In [0]:
pip install -e .

In [0]:
import gym
import gym_fraud_detection

In [0]:
env = gym.make('gym-fraud-detection-v0')

# Download proper libraries

In [0]:
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


In [0]:
# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Exploring the custom gym environment

In [0]:
print(env.action_space.n)

2


In [0]:
actions = []
rewards = []
count = 5

while True:
  action = env.action_space.sample()
  reward = env.step(action)
  actions.append(action)
  rewards.append(reward)
  count -= 1
  if count==0:
    break


In [0]:
for i in range(5):
  print("action : ", actions[i], "labelled data : ", env.label_for(i))
  print("reward : ", rewards[i])

action :  1 labelled data :  0.0
reward :  (1, -1, False, '{"true_positive_rate": 0, "false_positive_rate": 1, "true_negative_rate": 0, "false_negative_rate": 0}')
action :  1 labelled data :  0.0
reward :  (2, -1, False, '{"true_positive_rate": 0, "false_positive_rate": 2, "true_negative_rate": 0, "false_negative_rate": 0}')
action :  0 labelled data :  0.0
reward :  (3, 1, False, '{"true_positive_rate": 0, "false_positive_rate": 2, "true_negative_rate": 0, "false_negative_rate": 0}')
action :  0 labelled data :  0.0
reward :  (4, 1, False, '{"true_positive_rate": 0, "false_positive_rate": 2, "true_negative_rate": 0, "false_negative_rate": 0}')
action :  0 labelled data :  0.0
reward :  (5, 1, False, '{"true_positive_rate": 0, "false_positive_rate": 2, "true_negative_rate": 0, "false_negative_rate": 0}')


# Exploring data and creating pytorch model

link : https://www.kaggle.com/dakshmiglani/pytorch-credit-card-fraud-prediction-99-8

In [0]:
import torch, torch.nn as nn, torch.nn.functional as F
from torch.autograd import Variable
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch.utils.data as data_utils

In [0]:
df = pd.read_csv('./dataset/creditcard.csv')
df.head(1) # give us a sneek preview of the dataset xD

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.5516,-0.617801,-0.99139,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0


In [0]:
X = df.iloc[:, :-1].values 
y = df.iloc[:, -1].values

sc = StandardScaler()
X = sc.fit_transform(X)

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.1, random_state=1)

# Creating Deep Q-Network

## 1. Creating Experience Replay

In [0]:
from collections import deque

class Experience():
  def __init__(self, state, action, reward, next_state):
    self.state = state
    self.action = action
    self.reward = reward
    self.next_state = next_state

class ReplayMemory():
  def __init__(self, max_size=1000):
    self.buffer = deque(maxlen=max_size)
  
  def add(self, experience: Experience):
    self.buffer.append(experience)
  
  def sample(self, batch_size):
    idx = np.random.choice(np.arange(len(self.buffer)),
                           size=batch_size, 
                           replace=False)
    
    return [self.buffer[i] for i in idx]


## 2. Defining the Q-Network

In [0]:
class DQN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(30, 16)
        self.fc2 = nn.Linear(16, 18)
        self.fc3 = nn.Linear(18, 20)
        self.fc4 = nn.Linear(20, 24)
        # returning 2 q-values, one for every action
        self.fc5 = nn.Linear(24, 2)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.25)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.25)
        x = F.relu(self.fc3(x))
        x = F.dropout(x, p=0.25)
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [0]:
net = DQN().double()
net

DQN(
  (fc1): Linear(in_features=30, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=18, bias=True)
  (fc3): Linear(in_features=18, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=24, bias=True)
  (fc5): Linear(in_features=24, out_features=2, bias=True)
)

In [0]:
X_train = torch.from_numpy(X_train)
Y_train = torch.from_numpy(Y_train).double()

In [0]:
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# 3. Define the Agent


In [0]:
from agent import Agent

In [0]:
dqn_agent = Agent(nA=2)

# Training the model

## Optimize model using experience replay

In [0]:
train = data_utils.TensorDataset(X_train, Y_train)
train_loader = data_utils.DataLoader(train, batch_size=64, shuffle=True)

In [0]:
x = torch.tensor([[random.randrange(2)]], device=device, dtype=torch.long)
print(x)

tensor([[0]])


In [0]:
policy_net = DQN().to(device)

def get_epsilon_greedy_policy(state):
  if random.random()>0.9:
    with torch.no_grad():
      policy_net(state).max(1)[1].view(1, 1)
  else:
    torch.tensor([[random.randrange(2)]], device=device, dtype=torch.long)


In [0]:
for i in range(2):
  env.reset()
  accumulated_reward = 0
  for state_idx, data in enumerate(train_loader, 0):
    inputs, labels = data
    print(inputs.type)    
    action = get_epsilon_greedy_policy(inputs)
    print(action)

<built-in method type of Tensor object at 0x7f308ff5f750>
None
<built-in method type of Tensor object at 0x7f30906a71f8>


RuntimeError: ignored